## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
#Using a Neural Network classification for predicting market moves based on headlines

In [2]:
#link for data source -https://www.kaggle.com/aaron7sun/stocknews/downloads/stocknews.zip/1

In [3]:
# Your code here - remember to use markdown cells for comments as well!
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import statistics
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import accuracy_score 
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import np_utils
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from textblob import TextBlob 
%matplotlib inline
plt.style.use('ggplot')





Using TensorFlow backend.


In [4]:
data = pd.read_csv('./stocknews/Combined_News_DJIA.csv')
data.head()
data.tail()


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
1984,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
1985,2016-06-28,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",S&amp;P cuts United Kingdom sovereign credit r...,Huge helium deposit found in Africa,CEO of the South African state broadcaster qui...,"Brexit cost investors $2 trillion, the worst o...",Hong Kong democracy activists call for return ...,Brexit: Iceland president says UK can join 'tr...,...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
1986,2016-06-29,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,UK must accept freedom of movement to access E...,Devastated: scientists too late to captive bre...,British Labor Party leader Jeremy Corbyn loses...,A Muslim Shop in the UK Was Just Firebombed Wh...,Mexican Authorities Sexually Torture Women in ...,UK shares and pound continue to recover,...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
1987,2016-06-30,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,Boris Johnson says he will not run for Tory pa...,Six gay men in Ivory Coast were abused and for...,Switzerland denies citizenship to Muslim immig...,Palestinian terrorist stabs israeli teen girl ...,Puerto Rico will default on $1 billion of debt...,Republic of Ireland fans to be awarded medal f...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
1988,2016-07-01,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,"The president of France says if Brexit won, so...",British Man Who Must Give Police 24 Hours' Not...,100+ Nobel laureates 

In [5]:
data['Label'].value_counts()

1    1065
0     924
Name: Label, dtype: int64

In [6]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [7]:
batch_size = 32
nb_classes = 2
advancedvectorizer = TfidfVectorizer( min_df=0.04, max_df=0.3, max_features = 200000, ngram_range = (2, 2))
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
print(advancedtest.shape)
print(advancedtrain.shape)

(378, 401)
(1611, 401)


In [25]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

#print('X_train shape:', X_train.shape)
#print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.mean(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]


model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print("Training...")
model.fit(X_train, Y_train, nb_epoch=2, batch_size=16, validation_split=0.15)

print("Generating test predictions...")
preds14 = model.predict_classes(X_test, verbose=0)
acc14 = accuracy_score(test["Label"], preds14)


Training...


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1369 samples, validate on 242 samples
Epoch 1/2
1369/1369 [==============================] - 1s 463us/step - loss: 0.6949 - val_loss: 0.6828
Epoch 2/2
1369/1369 [==============================] - 0s 292us/step - loss: 0.6693 - val_loss: 0.6771
Generating test predictions...


In [26]:
acc14

0.5264550264550265

In [32]:
#Trying another model to improve accuracy
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
dataset = pd.read_csv('./stocknews/Combined_News_DJIA.csv')
# split into input (X) and output (y) variables
# define the keras model
model1 = Sequential()
model1.add(Dense(12, input_dim=401, activation='relu'))
model1.add(Dense(2, activation='relu'))
model1.add(Dense(2, activation='sigmoid'))
# compile the keras model
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model1.fit(X_train, Y_train, epochs=20, batch_size=10)
# evaluate the keras model
_, accuracy = model1.evaluate(X_train, Y_train)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/20
1611/1611 [==============================] - 1s 404us/step - loss: 0.6919 - acc: 0.5372
Epoch 2/20
1611/1611 [==============================] - 0s 150us/step - loss: 0.6876 - acc: 0.5419
Epoch 3/20
1611/1611 [==============================] - 0s 269us/step - loss: 0.6793 - acc: 0.5419
Epoch 4/20
1611/1611 [==============================] - 0s 157us/step - loss: 0.6658 - acc: 0.5422
Epoch 5/20
1611/1611 [==============================] - 0s 167us/step - loss: 0.6472 - acc: 0.5673
Epoch 6/20
1611/1611 [==============================] - 0s 167us/step - loss: 0.6214 - acc: 0.6778
Epoch 7/20
1611/1611 [==============================] - 0s 173us/step - loss: 0.5866 - acc: 0.7349
Epoch 8/20
1611/1611 [==============================] - 0s 247us/step - loss: 0.5497 - acc: 0.7486
Epoch 9/20
1611/1611 [==============================] - 0s 156us/step - loss: 0.5159 - acc: 0.7678
Epoch 10/20
1611/1611 [==============================] - 0s 203us/step - loss: 0.4851 - acc: 0.7914
Epoch 11/

In [77]:
# Predicting the Test set results
y_pred = model1.predict(X_test)
#y_pred = np.argmax(y_pred, axis=1)
#y_pred = (y_pred > 0.5)
#print(y_pred)
#print (Y_test)


In [76]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, Y_test)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multilabel-indicator targets